In [1]:
from pyspark.sql import SparkSession

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1590928980927_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Create spark session
spark = SparkSession \
    .builder \
    .appName("Sparkify") \
    .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Read in full sparkify dataset
event_data = "s3n://udacity-dsnd/sparkify/sparkify_event_data.json"
df = spark.read.json(event_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, IntegerType
from pyspark.sql.functions import desc,asc,isnull,when,count,col, sum as Fsum, max as Fmax
from pyspark.sql.functions import concat, col, lit
from pyspark.sql.functions import mean
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, Normalizer, StandardScaler, OneHotEncoderEstimator, StringIndexer
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, \
                DecisionTreeClassifier, GBTClassifier, LinearSVC
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder


import datetime

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df = df[df.userId!=1261737]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
flag_cancellation = udf(lambda x : 1 if x=="Cancellation Confirmation" else 0, IntegerType())
df = df.withColumn("churn",flag_cancellation("page"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
w1 = Window.partitionBy('userId')

# Create the cross-sectional dataframe that we'll use in analysis and modelling
df_user = df.select('userId','churn','gender','level') \
            .withColumn('churned_user',Fsum('churn').over(w1)) \
            .dropDuplicates(['userId']).drop('churn')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
indexer = StringIndexer(inputCol="gender",outputCol="genderIndex",handleInvalid="skip")
df_user = indexer.fit(df_user).transform(df_user)
indexer = StringIndexer(inputCol="level",outputCol="levelIndex")
df_user = indexer.fit(df_user).transform(df_user)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
encoder = OneHotEncoderEstimator(inputCols=["genderIndex", "levelIndex"],
                                 outputCols=["genderVector", "levelVector"])
model = encoder.fit(df_user)
df_user = model.transform(df_user)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
df_numSessions = df.select('userId','sessionId')\
                .dropDuplicates(['sessionId','userId'])\
                .withColumn('numSessions',count(df.userId).over(w1))\
                .dropDuplicates(['userId'])\
                .drop('sessionId')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
df_numSongs = df.select('userId','song')\
                .dropDuplicates(['userId','song'])\
                .withColumn('numSongs',count(df.userId).over(w1))\
                .dropDuplicates(['userId'])\
                .drop('song')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
df_numArtists = df.select('userId','artist')\
                .dropDuplicates(['userId','artist'])\
                .withColumn('numArtists',count(df.userId).over(w1))\
                .dropDuplicates(['userId'])\
                .drop('artist')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
df_playTime = df.filter(df['page']=='NextSong')\
            .select('userId','length')\
            .withColumn('playTime',Fsum('length').over(w1))\
            .dropDuplicates(['userId'])\
            .drop('song')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
df_activeDays = df.select('userId','ts','registration')\
    .withColumn('mostRecentTs',Fmax('ts').over(w1))\
    .dropDuplicates(['userId'])\
    .withColumn("activeDays",(col('ts')/1000.-col('registration')/1000.)/86400.)\
    .drop('registration')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
df_user = df_user.join(df_numSessions,df_user.userId==df_numSessions.userId).select(df_user["*"],df_numSessions['numSessions'])
df_user = df_user.join(df_numSongs,df_user.userId==df_numSongs.userId).select(df_user["*"],df_numSongs['numSongs'])
df_user = df_user.join(df_numArtists,df_user.userId==df_numArtists.userId).select(df_user["*"],df_numArtists['numArtists'])
df_user = df_user.join(df_playTime,df_user.userId==df_playTime.userId).select(df_user["*"],df_playTime['playTime'])
df_user = df_user.join(df_activeDays,df_user.userId==df_activeDays.userId).select(df_user["*"],df_activeDays['activeDays'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
w2 = Window.partitionBy('userId','page')
columns = [str(row.page) for row in df.select('page')\
           .dropDuplicates().sort('page').collect()]
df_pageVisits = df.select('userId','page')\
    .withColumn('pageVisits',count('userId').over(w2))\
    .groupby('userId')\
    .pivot('page',columns)\
    .mean('pageVisits')
df_pageVisits = df_pageVisits.na.fill(0).drop('Cancel','Cancellation Confirmation')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
df_inuse = df_pageVisits.join(df_user,df_pageVisits.userId==df_user.userId)\
.select(df_user.churned_user,df_pageVisits['*'],df_user.numSessions,df_user.numSongs,\
        df_user.numArtists, df_user.playTime,df_user.activeDays,df_user.genderIndex,\
        df_user.levelIndex,df_user.genderVector,df_user.levelVector)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
# Drop features that have correlation with each up
df_inuse= df_inuse.drop('Add Friend','Add to Playlist', 'Downgrade','Help','Home',\
                          'Logout','NextSong','Settings','Thumbs Up','Thumbs down','numSongs','numArtists',
                       'playTime')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# Vector Assembler
cols = df_inuse.drop('userID','churned_user').columns
assembler = VectorAssembler(inputCols=cols,outputCol='feature_vector')
df_inuse = assembler.transform(df_inuse).select('userId','churned_user','feature_vector')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# Standard Scaler
scaler = StandardScaler(inputCol='feature_vector',
                       outputCol ='features',
                       withStd=True)
scalerModel = scaler.fit(df_inuse)
df_inuse_scaled = scalerModel.transform(df_inuse)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
df_inuse_scaled = df_inuse_scaled.withColumnRenamed("churned_user", "label")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# prepare training and test data, sample by label
ratio = 0.7
train = df_inuse_scaled.sampleBy('churned_user',fractions = {0:ratio,1:ratio},
                                  seed = 42)
test = df_inuse_scaled.subtract(train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
def evaluate(model_name,train,test):
    evaluator = MulticlassClassificationEvaluator(metricName='f1')# paramGrid
    paramGrid = ParamGridBuilder() \
    .build()
    crossval = CrossValidator(estimator=model_name,
                          evaluator=evaluator, 
                          estimatorParamMaps=paramGrid,
                          numFolds=3)
    cvModel = crossval.fit(train)
    cvModel_metrics = cvModel.avgMetrics
    transformed_data=cvModel.transform(test)
    test_metrics = evaluator.evaluate(transformed_data)
    return (cvModel_metrics, test_metrics)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
a = evaluate(dtc,train,test)
print(a)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

([0.7170241472267253], 0.7339893416018917)
----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 56498)
Traceback (most recent call last):
  File "/usr/lib64/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib64/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib64/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib64/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/accumulators.py", line 266, in handle
    poll(authenticate_and_accum_updates)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/accumulators.py", line 254, in aut